In [1]:
import math
from tqdm import tqdm
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

In [2]:
from ucimlrepo import fetch_ucirepo 
from math import floor
from torch.utils.data import TensorDataset, DataLoader

def downloader(uci_id):
    # fetch dataset 
    uci_download = fetch_ucirepo(id=uci_id)
    
    # data (as pandas dataframes) 
    X_data = uci_download.data.features

    # Drop category and date variables
    if uci_id == 1:
        X_data = X_data.drop(["Sex"], axis=1)
    if uci_id == 275:
        X_data = X_data.drop(["dteday"], axis=1)

    y_data = uci_download.data.targets
    y = y_data.squeeze()

    X = torch.tensor(X_data.values, dtype=torch.float32)
    y = torch.tensor(y.values, dtype=torch.float32)

    train_n = int(floor(0.8 * len(X)))
    train_x = X[:train_n, :].contiguous()
    train_y = y[:train_n].contiguous()

    test_x = X[train_n:, :].contiguous()
    test_y = y[train_n:].contiguous()

    # Create TensorDataset and DataLoader for training and test sets
    train_dataset = TensorDataset(train_x, train_y)
    test_dataset = TensorDataset(test_x, test_y)

    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

    return train_x, train_y, test_x, test_y, train_loader, test_loader

def whole_process(train_x, train_y, test_x, test_y, train_loader, test_loader):

    data_dim = train_x.size(-1)

    class LargeFeatureExtractor(torch.nn.Sequential):
        def __init__(self):
            super(LargeFeatureExtractor, self).__init__()
            self.add_module('linear1', torch.nn.Linear(data_dim, 1000))
            self.add_module('relu1', torch.nn.ReLU())
            self.add_module('linear2', torch.nn.Linear(1000, 500))
            self.add_module('relu2', torch.nn.ReLU())
            self.add_module('linear3', torch.nn.Linear(500, 50))
            self.add_module('relu3', torch.nn.ReLU())
            self.add_module('linear4', torch.nn.Linear(50, 2))

    feature_extractor = LargeFeatureExtractor()
    from gpytorch.models import ApproximateGP
    from gpytorch.variational import CholeskyVariationalDistribution
    from gpytorch.variational import VariationalStrategy

    class GPModel(ApproximateGP):
        def __init__(self, inducing_points, feature_extractor):

            variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))

            variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)

            super(GPModel, self).__init__(variational_strategy)
            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.SpectralMixtureKernel(num_mixtures=2, ard_num_dims=2))
            # self.covar_module = gpytorch.kernels.ScaleKernel(
            #     gpytorch.kernels.RBFKernel(
            #         lengthscale_prior=gpytorch.priors.SmoothedBoxPrior(
            #             math.exp(-1), math.exp(1), sigma=0.1, transform=torch.exp
            #         )
            #     )
            # )
            self.feature_extractor = feature_extractor
            self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-1., 1.)

        def forward(self, x):
            projected_x = self.feature_extractor(x)
            projected_x = self.scale_to_bounds(projected_x)
            mean_x = self.mean_module(projected_x)
            covar_x = self.covar_module(projected_x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    inducing_points = train_x[:1, :]
    model = GPModel(inducing_points=inducing_points, feature_extractor=feature_extractor)
    likelihood = gpytorch.likelihoods.GaussianLikelihood()


    num_epochs = 100
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam([
        #{'params': model.feature_extractor.parameters()},
        #{'params': model.covar_module.parameters()},
        #{'params': model.mean_module.parameters()},
        #{'params': model.variational_strategy.parameters()},
        {'params': likelihood.parameters()},
        {'params': model.parameters()}
    ], lr=0.01)
    # Our loss object. We're using the VariationalELBO
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))


    epochs_iter = tqdm(range(num_epochs), desc="Epoch")
    for i in epochs_iter:
        # Within each iteration, we will go over each minibatch of data
        minibatch_iter = tqdm(train_loader, desc="Minibatch", leave=False)
        for x_batch, y_batch in minibatch_iter:
            optimizer.zero_grad()
            output = model(x_batch)
            loss = -mll(output, y_batch)
            minibatch_iter.set_postfix(loss=loss.item())
            loss.backward()
            optimizer.step()

    model.eval()
    likelihood.eval()
    means = torch.tensor([0.])
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            preds = model(x_batch)
            means = torch.cat([means, preds.mean.cpu()])
    means = means[1:]
    print('Test MAE: {}'.format(torch.mean(torch.abs(means - test_y.cpu()))))

    means = torch.tensor([0.])
    with torch.no_grad():
        for x_batch, y_batch in train_loader:
            preds = model(x_batch)
            means = torch.cat([means, preds.mean.cpu()])
    means = means[1:]
    print('Train MAE: {}'.format(torch.mean(torch.abs(means - train_y.cpu()))))

experiment_datasets = [1, 275, 477]
# abalone, bike share, real estate

for experiment in experiment_datasets:
    print("Experiment " + str(experiment))
    train_x, train_y, test_x, test_y, train_loader, test_loader = downloader(experiment)
    whole_process(train_x, train_y, test_x, test_y, train_loader, test_loader)

Experiment 1


Epoch: 100%|██████████| 100/100 [00:33<00:00,  2.99it/s]


Test MAE: 1.9252406358718872
Train MAE: 2.5557947158813477
Experiment 275


Epoch: 100%|██████████| 100/100 [02:10<00:00,  1.30s/it]


Test MAE: 56.53175354003906
Train MAE: 172.42398071289062
Experiment 477


Epoch: 100%|██████████| 100/100 [00:04<00:00, 22.38it/s]

Test MAE: 15.104546546936035
Train MAE: 15.448212623596191
